In [1]:
import pandas as pd
import numpy as np
import MySQLdb

In [2]:
Anio='2018'
Mes='12'
Dia_act='14'

In [3]:
fecha_ini='2018-12-01 00:00:00'
fecha_fin='2018-12-31 21:00:00'

In [4]:
db = MySQLdb.connect(host="172.18.55.6",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="SISTEMECUADOR_ATM")        # el nombre de la base de datos

In [5]:
consulta="""
SELECT 
    NRO_IDENTIFICACION_CLIENTE, COD_AGENTE,
    CASE CONTACTO_GESTIONO
                WHEN 'CLIENTE' THEN 1
                WHEN 'TERC. VALIDO' THEN 2
                WHEN 'TERC. NO VALIDO' THEN 3
                WHEN 'NO CONTACTO' THEN 4
                ELSE 5
            END AS CONTACTO,
	CASE RESPUESTA_OBTENIDA
                WHEN 'ACUERDO DE PAGO' THEN 1
                WHEN 'ACUERDO DE CONVENIO' THEN 2
                WHEN 'LOCALIZADO SIN ACUERDO' THEN 3
                ELSE 4
            END AS TIPO_ACUERDO
FROM
    sistemecuador_atm.gestion
WHERE
    ACCION_REALIZADA = 'HACER LLAMADA'
AND
FECHA_GESTION BETWEEN '{0}' AND '{1}'
AND
COMENTARIOS_GESTIONO NOT LIKE '%CAMPAÑA CONVENIO%'
;
""".format(fecha_ini,fecha_fin)

In [6]:
resultado=pd.read_sql(consulta, con=db)
resultado = resultado.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'ID'})
contactabilidad=resultado.groupby('ID').agg({'CONTACTO':'min','TIPO_ACUERDO':'min'})
contactabilidad['CONTACTO'].replace(to_replace=[1,2,3,4,5],value=['TITULAR','TERCERO','EQUIVOCADO','NO CONTACTO','SIN GESTION'],inplace=True)
contactabilidad['TIPO_ACUERDO'].replace(to_replace=[1,2,3,4],value=['AC. PAGO','AC. CONVENIO','LOCALIZADO','OTROS'],inplace=True)

In [7]:
consulta_ivr="""
    SELECT NRO_IDENTIFICACION_CLIENTE as ID,
        RESPUESTA_OBTENIDA
    FROM
        sistemecuador_atm.gestion_ivr
    WHERE
        FECHA_GESTION BETWEEN '{0}' AND '{1}';
""".format(fecha_ini,fecha_fin)

In [8]:
resultado_ivr=pd.read_sql(consulta_ivr, con=db)

In [9]:
resultado_ivr.head()

,ID,RESPUESTA_OBTENIDA
0,1000005262,CONTESTA IVR
1,1001296837,CONTESTA IVR
2,1001527017,CONTESTA IVR
3,1002339149,CONTESTA IVR
4,1002472304,CONTESTA IVR


In [10]:
resultado_ivr["RESPUESTA_OBTENIDA"] = resultado_ivr["RESPUESTA_OBTENIDA"].astype('category')
resultado_ivr["RESPUESTA_OBTENIDA"]  = resultado_ivr["RESPUESTA_OBTENIDA"].cat.set_categories(["CONTESTA IVR","CONTESTO IVR","SMS ENVIADO","NO CONTESTO IVR","NO CONTESTA IVR","SIN IVR"])

In [11]:
resultado_ivr.head()

,ID,RESPUESTA_OBTENIDA
0,1000005262,CONTESTA IVR
1,1001296837,CONTESTA IVR
2,1001527017,CONTESTA IVR
3,1002339149,CONTESTA IVR
4,1002472304,CONTESTA IVR


In [12]:
resultado_ivr_ordenada=resultado_ivr.sort_values(by=['RESPUESTA_OBTENIDA'],ascending=[True])

In [13]:
resultado_sinduplicados=resultado_ivr_ordenada.drop_duplicates(['ID'], keep='first')

In [14]:
# writer = pd.ExcelWriter('..\..\CRUZAR NUMEROS\BASES PROVAR IVRS.xlsx', engine='xlsxwriter')

# resultado_ivr.to_excel(writer, sheet_name='BASE TOTAL')
# worksheet = writer.sheets['BASE TOTAL']


# resultado_ivr_ordenada.to_excel(writer, sheet_name='BASE CORRIENTE')
# worksheet = writer.sheets['BASE CORRIENTE']

# resultado_sinduplicados.to_excel(writer, sheet_name='BASE MENORES')
# worksheet = writer.sheets['BASE MENORES']

# writer.save()

In [15]:
#nombre de los archivos Asignacion normal
nom_actual_cor='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2018\\DICIEMBRE\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_cor='Asignacion'
cartera_cor='CORRIENTE'

# nombre de los archivos Menores
nom_actual_men='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2018\\DICIEMBRE\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_men='Menores'
cartera_men='MENORES'

In [16]:
def agregar_ceros(base):
    ced=base['IDENTIFICACION_DEUDOR']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [17]:
def verificar_tipo(base):
    ced=base['ID']
    if len(ced)==13:
        return 'RUC'
    return 'CED'

In [18]:
def ultimo_digito(base):
    placa=base['PLACA_VEHICULO']
    placa=str(placa)
    if (placa == ''):
        return '10'
    if placa[-1].isdigit():
        return placa[-1]
    return '10'

In [19]:
#leer archivo excel asignacion actual y anterior
asignacion_actual_cor=pd.read_excel(open(nom_actual_cor,'rb'), sheet_name=nombre_hoja_cor, dtype={"IDENTIFICACION_DEUDOR": str})
#leer archivo excel asignacion actual y anterior
asignacion_actual_men=pd.read_excel(open(nom_actual_men,'rb'), sheet_name=nombre_hoja_men, dtype={"IDENTIFICACION_DEUDOR": str})

In [20]:
asignacion_actual_cor[asignacion_actual_cor['SEGUNDO_APELLIDO']=='NUNEZ']='NUÑEZ'

In [21]:
asignacion_actual_cor= asignacion_actual_cor.fillna('')
asignacion_actual_cor['ID']=asignacion_actual_cor.apply(agregar_ceros,axis=1)
asignacion_actual_cor['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)
try:
    asignacion_actual_cor['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
except:
    asignacion_actual_cor['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
asignacion_actual_cor['NOMBRE']=asignacion_actual_cor['RAZONSOCIAL']+' '+asignacion_actual_cor['PRIMER_APELLIDO']+' '+asignacion_actual_cor['SEGUNDO_APELLIDO']+' '+asignacion_actual_cor['PRIMER_NOMBRE']
asignacion_actual_cor['CARTERA']='CORRIENTE'

In [22]:
asignacion_actual_cor['TIPO']=asignacion_actual_cor.apply(verificar_tipo,axis=1)
asignacion_actual_cor['U_PLACA']=asignacion_actual_cor.apply(ultimo_digito,axis=1)

In [23]:
cruce_corr=pd.merge(asignacion_actual_cor,contactabilidad,on=['ID'],how='left')

In [24]:
cruce_corr= cruce_corr.fillna('')

In [25]:
cruce_corr_ivr=pd.merge(cruce_corr,resultado_sinduplicados,on=['ID'],how='left')
cruce_corr_ivr= cruce_corr_ivr.fillna('SIN IVR')

In [26]:
cruce_corr_ivr.head()

,IDENTIFICACION_DEUDOR,TIPO_IDENTIFICACION,NUMERO_CUENTA,ESTADO AXIS,ESTADO_CREDITO,SALDO,DIAS_MORA,ESTADO,CODIGO_CONDICION,PLACA_VEHICULO,...,MEJOR_RESPUESTA,FECHA_COACTIVO,ID,NOMBRE,CARTERA,TIPO,U_PLACA,CONTACTO,TIPO_ACUERDO,RESPUESTA_OBTENIDA
0,I685506,CED,5850416,PENDIENTE,COBRAR,57.9,20,ASISTENCIA TELEFONIC,1,OAN0814,...,,NO COACTIVA,I685506,MOREJON RODRIGUEZ YOEL,CORRIENTE,CED,4,,,SIN IVR
1,1890153999001,NUM,5771511,PENDIENTE,COBRAR,115.8,40,LOCALIZADO,2,TBG2380,...,CONTESTA IVR,NO COACTIVA,1890153999001,EMPRESA DE TRANSPORT,CORRIENTE,RUC,0,NO CONTACTO,OTROS,CONTESTA IVR
2,1890069963001,NUM,5870866,PENDIENTE,COBRAR,115.8,15,PROMESA ROTA,2,TBD7440,...,,NO COACTIVA,1890069963001,AMBASEG AMBATO SEGURIDAD C LTDA,CORRIENTE,RUC,0,,,CONTESTA IVR
3,1801712058001,CED,5857095,PENDIENTE,COBRAR,115.8,21,PROMESA ROTA,2,GBO2848,...,NO CONTESTA IVR,NO COACTIVA,1801712058001,LUIS HERNAN GARZON URRUTIA,CORRIENTE,RUC,8,TITULAR,LOCALIZADO,CONTESTA IVR
4,1792734266001,NUM,5848663,PENDIENTE,COBRAR,57.9,20,ASISTENCIA TELEFONIC,1,PDC6285,...,,NO COACTIVA,1792734266001,PUBLICIDAD Y LOGISTICA JATIVA REASCOS DEFAZ DJ...,CORRIENTE,RUC,5,,,SIN IVR


In [28]:
try:
    base_gestion_cor=cruce_corr_ivr[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
except:
    base_gestion_cor=cruce_corr_ivr[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [29]:
ordenada_cor.count()

FECHA_COACTIVO  min    14329
SALDO           sum    14329
DIAS_MORA       min    14329
                max    14329
U_PLACA         min    14329
dtype: int64

In [31]:
asignacion_actual_men= asignacion_actual_men.fillna('')
asignacion_actual_men['ID']=asignacion_actual_men.apply(agregar_ceros,axis=1)
asignacion_actual_men['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)
try:
    asignacion_actual_men['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
except:
    asignacion_actual_men['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
asignacion_actual_men['NOMBRE']=asignacion_actual_men['RAZONSOCIAL']+' '+asignacion_actual_men['PRIMER_APELLIDO']+' '+asignacion_actual_men['SEGUNDO_APELLIDO']+' '+asignacion_actual_men['PRIMER_NOMBRE']
asignacion_actual_men['CARTERA']='MENORES'

In [32]:
asignacion_actual_men['TIPO']=asignacion_actual_men.apply(verificar_tipo,axis=1)
asignacion_actual_men['U_PLACA']=asignacion_actual_men.apply(ultimo_digito,axis=1)

In [33]:
cruce_men=pd.merge(asignacion_actual_men,contactabilidad,on=['ID'],how='left')

In [34]:
cruce_men= cruce_men.fillna('')

In [35]:
cruce_men_ivr=pd.merge(cruce_men,resultado_sinduplicados,on=['ID'],how='left')
cruce_men_ivr= cruce_men_ivr.fillna('SIN IVR')

In [36]:
try:
    base_gestion_men=cruce_men_ivr[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']]
    ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
except:
    base_gestion_men=cruce_men_ivr[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']]
    ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','CONTACTO','TIPO_ACUERDO','RESPUESTA_OBTENIDA']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [37]:
ordenada_men.count()

FECHA_COACTIVO  min    21639
SALDO           sum    21639
DIAS_MORA       min    21639
                max    21639
U_PLACA         min    21639
dtype: int64

In [38]:
ordenada_men.head()

FECHA_COACTIVO  \
                                                                                                                                                   min   
IDENTIFICACION_DEUDOR ID         NOMBRE                   SUBCAMPANAPORCLIENTE CARTERA TIPO CONTACTO    TIPO_ACUERDO RESPUESTA_OBTENIDA                  
100025030             0100025030  CRESPO DELGADO JULIO    MULTAS NUEVAS        MENORES CED                           CONTESTA IVR          NO COACTIVA   
1000584647            1000584647  DELGADO CRIBAN NORBERTO MULTAS NUEVAS        MENORES CED                           CONTESTA IVR          NO COACTIVA   
1000620623            1000620623  REVELO JOSE ELIAS       MULTAS NUEVAS        MENORES CED                           CONTESTA IVR          NO COACTIVA   
1000729788            1000729788  LOZA ESPINOSA JULIO     MULTAS NUEVAS        MENORES CED  NO CONTACTO OTROS        CONTESTA IVR                43088   
1000779148            1000779148  RUBIO PALACIOS FRANKLIN MULTAS NUEVAS        MENORES CED                           CONTESTA IVR          NO COACTIVA   

                                                                                                                                        SALDO  \
                                                                                                                                          sum   
IDENTIFICACION_DEUDOR ID         NOMBRE                   SUBCAMPANAPORCLIENTE CARTERA TIPO CONTACTO    TIPO_ACUERDO RESPUESTA_OBTENIDA         
100025030             0100025030  CRESPO DELGADO JULIO    MULTAS NUEVAS        MENORES CED                           CONTESTA IVR        19.3   
1000584647            1000584647  DELGADO CRIBAN NORBERTO MULTAS NUEVAS        MENORES CED                           CONTESTA IVR        57.9   
1000620623            1000620623  REVELO JOSE ELIAS       MULTAS NUEVAS        MENORES CED                           CONTESTA IVR        38.6   
1000729788            1000729788  LOZA ESPINOSA JULIO     MULTAS NUEVAS        MENORES CED  NO CONTACTO OTROS        CONTESTA IVR        57.9   
1000779148            1000779148  RUBIO PALACIOS FRANKLIN MULTAS NUEVAS        MENORES CED                           CONTESTA IVR        38.6   

                                                                                                                                        DIAS_MORA  \
                                                                                                                                              min   
IDENTIFICACION_DEUDOR ID         NOMBRE                   SUBCAMPANAPORCLIENTE CARTERA TIPO CONTACTO    TIPO_ACUERDO RESPUESTA_OBTENIDA             
100025030             0100025030  CRESPO DELGADO JULIO    MULTAS NUEVAS        MENORES CED                           CONTESTA IVR             229   
1000584647            1000584647  DELGADO CRIBAN NORBERTO MULTAS NUEVAS        MENORES CED                           CONTESTA IVR             209   
1000620623            1000620623  REVELO JOSE ELIAS       MULTAS NUEVAS        MENORES CED                           CONTESTA IVR              66   
1000729788            1000729788  LOZA ESPINOSA JULIO     MULTAS NUEVAS        MENORES CED  NO CONTACTO OTROS        CONTESTA IVR             174   
1000779148            1000779148  RUBIO PALACIOS FRANKLIN MULTAS NUEVAS        MENORES CED                           CONTESTA IVR             314   

                                                                                                                                              \
                                                                                                                                         max   
IDENTIFICACION_DEUDOR ID         NOMBRE                   SUBCAMPANAPORCLIENTE CARTERA TIPO CONTACTO    TIPO_ACUERDO RESPUESTA_OBTENIDA        
100025030             0100025030  CRESPO DELGADO JULIO    MULTAS NUEVAS        MENORES CED                           

In [39]:
base_total=pd.concat([ordenada_cor, ordenada_men])

In [40]:
nuevo_archivo='BASE ARREGLADA DICIEMBRE.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
base_total.to_excel(writer,sheet_name='BASE_TOTAL')
worksheet = writer.sheets['BASE_TOTAL']
writer.save()